In [1]:

import os
import pty
import subprocess
import pprint
import time
import json
# https://github.com/helgefmi/ohno/blob/master/ohno/client/pty.py ## loosely based on this code
class Nethack:
    def __init__(self):
        self.master = None
        self.slave = None
        self.process = None

    def start_process(self):
        self.master, self.slave = pty.openpty()
        self.process = subprocess.Popen(
            ["nethack"],
            stdin=self.slave,
            stdout=self.slave,
            stderr=self.slave,
            close_fds=True, # this line comes from Bing Chat and I don't know what it does really
            universal_newlines=True,
        )
        os.close(self.slave) # Copilot added this so take it with a grain of salt...if it works, we don't need to save the reference

    def read_output(self, sleep = 0.25, chars = 1024, print_output = True):
    #def read_output(self, sleep = 0.25, chars = 8192, print_output = True):
        if self.process is None:
            raise ValueError("Process not started")

        if sleep is not None and sleep > 0:
            time.sleep(sleep)

        output = os.read(self.master, chars).decode()
        if print_output:
            pprint.pprint(output)

        return output

    def send_command(self, command):
        os.write(self.master, command.encode() + b"\n")

    def close(self):
        try:
            self.process.terminate()
            os.close(self.master)
        except:
            pass

    @staticmethod
    def clean_locks():
        password = json.load(open("private.json"))["sudo"]
        os.system(f'echo "{password}" | sudo -S rm -f /var/games/nethack/*lock*')

# 24, 80

In [2]:
import stransi
import re

class Terminal:
    unsupported = {
        r"\x1b7": "save cursor",
        r"\x1b8": "restore cursor",
        r"\x1b\[\?47h": "set alternate screen",
        r"\x1b\)0": "set to ASCII",
    }

    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.reset()

    def reset(self):
        self.characters = [[' ' for i in range(self.cols)] for _ in range(self.rows)]
        self.foregrounds = [["white" for i in range(self.cols)] for _ in range(self.rows)]
        self.backgrounds = [["black" for i in range(self.cols)] for _ in range(self.rows)] # does Nethack actually do this?

    def text(self):
        s = ""
        for i in range(self.rows):
            for j in range(self.cols): # let's ignore colors for now
                s += self.characters[i][j]
            s += "\n"
        return s
    
    def print(self):
        print(self.text())
                
    def update(self, input):
        row, col = 0, 0
        self.reset()
        # this is where the logic is going to get complex
        cleaned = input
        for k, v in Terminal.unsupported.items():
            rgx = re.compile(k)
            cleaned = re.sub(rgx, "", cleaned)
        ansi = stransi.Ansi(cleaned)
        for instr in ansi.instructions():
            print(instr) # for now
            if isinstance(instr, str):
                for i in range(len(instr)):
                    ch = instr[i]
                    if ch == "\r":
                        continue
                    try:
                        self.characters[row][col] = ch
                        col += 1
                    except:
                        print(f"something out of bounds: col={col}, row={row}, i={i} len={len(instr)}")
            elif isinstance(instr, stransi.SetCursor):
                row = instr.move.x
                col = instr.move.y 
            else:
                print("unhandled instruction")
                print(instr)


In [3]:
Nethack.clean_locks()
nethack = Nethack()
nethack.start_process()
text1 = nethack.read_output()
nethack.send_command("y")
text2 = nethack.read_output()
nethack.send_command(" ")
text3 = nethack.read_output()

[sudo] password for perplexity: 

('\x1b7\x1b[?47h\x1b)0\x1b[H\x1b[2J\x1b[H\x1b[2;1HNetHack, Copyright '
 '1985-2020\r'
 '\x1b[3;1H         By Stichting Mathematisch Centrum and M. Stephenson.\r'
 '\x1b[4;1H         Version 3.6.6 Unix, revised Feb 25 14:00:45 2021.\r'
 '\x1b[5;1H         See license for details.\r'
 "\x1b[6;1H\x1b[7;1H\x1b[8;1HShall I pick character's race, role, gender and "
 'alignment for you? [ynaq] ')
('y\x1b[H\x1b[2J\x1b[H       name: perplexity\r'
 '\x1b[2;1H       role: Rogue\r'
 '\x1b[3;1H       race: human\r'
 '\x1b[4;1H     gender: female\r'
 '\x1b[5;1H  alignment: chaotic\x1b[1;40H\x1b[K Is this ok? '
 '[ynq]\x1b[2;40H\x1b[K \x1b[3;40H\x1b[K perplexity, chaotic female human '
 'Rogue\x1b[4;40H\x1b[K \x1b[5;40H\x1b[K y + Yes; start game\x1b[6;40H\x1b[K n '
 '- No; choose role again\x1b[7;40H\x1b[K q - Quit\x1b[8;40H\x1b[K\x1b[C(end) '
 '\x1b[1;39H\x1b[K\x1b[2;39H\x1b[K\x1b[3;39H\x1b[K\x1b[4;39H\x1b[K\x1b[5;39H\x1b[K\x1b[6;39H\x1b[K\x1b[7;39H\x1b[K\x1b[8;39H\x1b[K\x1b[9;39H\x1b[K\x1b[H\x1b[2J

In [10]:
term = Terminal(24, 80)
term.update(text2)

y
SetCursor(move=CursorMove(x=0, y=0, relative=False))
SetClear(region=<Clear.SCREEN: 2>)
unhandled instruction
SetClear(region=<Clear.SCREEN: 2>)
SetCursor(move=CursorMove(x=0, y=0, relative=False))
       name: perplexity
SetCursor(move=CursorMove(x=1, y=0, relative=False))
       role: Rogue
SetCursor(move=CursorMove(x=2, y=0, relative=False))
       race: human
SetCursor(move=CursorMove(x=3, y=0, relative=False))
     gender: female
SetCursor(move=CursorMove(x=4, y=0, relative=False))
  alignment: chaotic
SetCursor(move=CursorMove(x=0, y=39, relative=False))
SetClear(region=<Clear.LINE_AFTER: 3>)
unhandled instruction
SetClear(region=<Clear.LINE_AFTER: 3>)
 Is this ok? [ynq]
SetCursor(move=CursorMove(x=1, y=39, relative=False))
SetClear(region=<Clear.LINE_AFTER: 3>)
unhandled instruction
SetClear(region=<Clear.LINE_AFTER: 3>)
 
SetCursor(move=CursorMove(x=2, y=39, relative=False))
SetClear(region=<Clear.LINE_AFTER: 3>)
unhandled instruction
SetClear(region=<Clear.LINE_AFTER: 3>)
 p

In [11]:
print(term.text()) # I see...so there are at least two issues here:
# 1) It seems like we navigated through several screens using just one command...could I perhaps be requesting too much data?
# 2) We're not handling SetClear or SetAttribute instructions

Perplexity the Footpad It is written in the Book of Kos:Wi:15 Ch:11 Chaotic    [
(end)  role: Rogue                                                              
       race: human         After the Creation, the cruel god Moloch rebelledue  
     gender: female        against the authority of Marduk the Creator.         
  alignment: chaotic       Moloch stole from Marduk the most powerful of all    
                           t            -----+---------role again               
                                        q - Quit                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                            